In [27]:
from src.data import CICIDS2017
from src.models.API import Mistral, Gemini
from src.pipelines import TAPipeline
from src.pipelines.utils import plot_accuracies

import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [28]:
dataset = CICIDS2017(classes_mapping=True, pca=False)
dataset.load()

2025-03-29 00:10:33,257 - INFO - Loading dataset...


In [29]:
dataset.balance_(n=45, category_col="Label")
icl_data, test_data = dataset.train_test_split(test_size=0.15)
x_test = test_data.drop(columns=["Label"])
y_test = test_data["Label"]
print(icl_data.shape, test_data.shape)

(307, 71) (55, 71)


In [30]:
chat = Gemini(model="gemini-2.0-pro-exp-02-05")
# chat = Mistral()
chat.ask("hey")

2025-03-29 00:10:46,721 - INFO - AFC is enabled with max remote calls: 10.
2025-03-29 00:10:52,003 - INFO - AFC remote call 1 is done.


{'answer': ['Hey there! How can I help you today?'], 'finish_reason': ['STOP']}

In [ ]:
chat_pl = TAPipeline(chat)
n = 3
chat_results = chat_pl.evaluate(
    icl_data,
    x_test,
    y_test,
    "Behaving like a Intrusion Detection System, classifing the data based on the PC features in the possible classes of attacks or not attack (BENIGN)",
    "Label",
    num_predictions=n,
    other_info=dataset._get_id()
)

2025-03-29 00:10:52,069 - INFO - Starting predictions...asking the model...
2025-03-29 00:10:52,075 - INFO - Prediction nº1...
2025-03-29 00:10:52,077 - INFO - AFC is enabled with max remote calls: 10.
2025-03-29 00:11:41,281 - INFO - AFC remote call 1 is done.
2025-03-29 00:11:41,290 - INFO - Response: ['Infiltration', 'Infiltration', 'Infiltration']
2025-03-29 00:11:41,291 - INFO - Prediction nº2...
2025-03-29 00:11:41,293 - INFO - AFC is enabled with max remote calls: 10.
2025-03-29 00:12:41,626 - INFO - AFC remote call 1 is done.
2025-03-29 00:12:41,628 - INFO - Response: ['Bot', 'Bot', 'Bot']
2025-03-29 00:12:41,628 - INFO - Prediction nº3...
2025-03-29 00:12:41,629 - INFO - AFC is enabled with max remote calls: 10.
2025-03-29 00:13:07,098 - INFO - AFC remote call 1 is done.
2025-03-29 00:13:07,099 - INFO - Response: ['BENIGN', 'BENIGN', 'BENIGN']
2025-03-29 00:13:07,099 - INFO - Prediction nº4...
2025-03-29 00:13:07,100 - INFO - AFC is enabled with max remote calls: 10.
2025-03-2

In [ ]:
chat_results["accuracy"]

1.0

In [ ]:
accuracy_plot = plot_accuracies([chat_results["accuracy"]], [chat.name + " - " + f"majority voting with {n} predictions"])
# accuracy_plot = plot_accuracies([chat_results["accuracy"]], [chat.name])